# Pillar 4 · Auditability and Traceability

Apply structured logging and provenance tracking to make agent decisions explainable and auditable.

## Learning Objectives

- Capture detailed interaction logs for internal and external audits
- Maintain lineage between prompts, tools, and outputs
- Package evidence for compliance attestations

## Traceability Framework

1. **Structured Logs**: Extend message formatting utilities (see `agent_utils.format_messages`) to include policy decision tags and evaluation scores.
2. **Conversation Snapshots**: Persist per-run transcripts in Cosmos DB using `AgentDB.create_agent` metadata fields for quick retrieval.
3. **Explainability Records**: Store agent rationales and tool call metadata to support root cause analysis during audits.
4. **Evidence Bundles**: Automate generation of audit packages for regulators and internal review boards.

## Audit Evidence Checklist

- End-to-end interaction trace captured
- Tool invocation history preserved
- Explanation artifacts linked to decisions

## Configure PATH for Azure CLI

Ensure the Azure CLI is accessible in the notebook kernel's PATH.

In [ ]:
import os
import shutil

# Replace with the directory you want to add
new_path_entry = "/opt/homebrew/bin"
current_path = os.environ.get('PATH', '')

if new_path_entry not in current_path.split(os.pathsep):
    os.environ['PATH'] = new_path_entry + os.pathsep + current_path
    print(f"Updated PATH for this session: {os.environ['PATH']}")
else:
    print(f"PATH already contains {new_path_entry}: {current_path}")

# You can then verify with shutil.which again
print(f"Location of 'az' found by kernel now: {shutil.which('az')}")

In [ ]:
# Retrieve a transcript and enrich it with governance tags
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'utils'))
from agent_utils import AgentManager, format_messages

# Load environment variables
load_dotenv("../.env")


print("✅ Imports successful")

endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
if not endpoint:
    raise ValueError(
        "Set AZURE_AI_PROJECT_ENDPOINT in .env.local before running this notebook.")

project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

manager = AgentManager(project_client)

thread_id = os.getenv("TARGET_THREAD_ID", "thread_05h8r1L98jtLijZvEVB66lnF")
if not thread_id:
    raise ValueError("Set TARGET_THREAD_ID to the thread you want to audit.")

messages = manager.get_messages(thread_id=thread_id)
formatted = format_messages(messages)
print(formatted)

In [ ]:
# Persist an audit bundle to disk
import json
from datetime import datetime

bundle = []
for message in messages:
    texts = []
    for content in getattr(message, "text_messages", []):
        if hasattr(content, "text") and hasattr(content.text, "value"):
            texts.append(content.text.value)
        else:
            texts.append(str(content))
    
    # Convert timestamp to ISO format string for JSON serialization
    timestamp = getattr(message, "created_at", None)
    if timestamp is not None and hasattr(timestamp, 'isoformat'):
        timestamp = timestamp.isoformat()
    
    entry = {
        "id": getattr(message, "id", None),
        "role": getattr(message, "role", None),
        "timestamp": timestamp,
        "text": texts
    }
    bundle.append(entry)

output_dir = Path("governance_exports")
output_dir.mkdir(exist_ok=True)
output_path = output_dir / f"{thread_id}_audit_{datetime.utcnow().isoformat()}.json"

with output_path.open("w", encoding="utf-8") as handle:
    json.dump(bundle, handle, indent=2)

print(f"Audit bundle written to {output_path}")

## Next Notebook

Continue with `06_stakeholder_engagement.ipynb` to coordinate governance stakeholders.